# Day 4 - Let's play bingo

How curious.  My first job after graduating university was to program touchscreen bingo systems, so I'll be fascinated to see how accurate this is.  My first note is that the bingo cards are 5x5, whereas good british cards are 9x3 or 4x4.  So it's american bingo then.

This one is going to be the first opportunity for us to consider some form of data structure I think.  Previously we've worked on sets of numbers, and there's been no real need for a more complex data structure at all.

But in this case, we're going to load a stream of random numbers, and then we're going to have to store a number of boards of 5x5 bingo cards.  We're then going to have to step through the number list, and for each number called, we're going to go through our boards and mark the number (dab them in Bingo parlance FYI), and then we're also going to have to check for a winning card, one in this case that has a line of 5 in a row.  In this case, a board is winning if there's a row or a column of marked numbers.

We also need that board to keep track of which number is marked and which are not in order to calculate the final sum for the board.  This feels like it calls for a class.

We can have a board class.  It can keep the 25 numbers on the board in one list, and a list of booleans indicating whether or not a number is marked seperately.  We could use a multidimensional list, so something like `numbers = [[1,2,3,4,5],[6,7,8,9,10],[11,12,13,14,15]]`, but I suspect it's easier to use a bit of math on a simple list of numbers.
If we want to check a column, we can simply look at every 5th number, offset by the column number, so at numebr 0,5,10 for 1st column or number 1,6,11 for the second column.  This is easy enough to do.

Oh, finally we need to be able to parse the data into our boards.  That's a lot more complex than previous parsing, so we're going to want to consume the input file in two chunks.  Firstly the first line is just the list of numbers called.  Then we can skip a line, and line 3 onwards can be seen as a set of 5 lines with a board, and skip a line.  We can repeat that to get all the boards.

Onwards

In [1]:
## Import ipytest and get it setup for use in Python Notebook
import pytest
import ipytest
ipytest.autoconfig()

In [2]:
class Board:
    def __init__(self, numbers):
        self.numbers = numbers
        self.marked = [False for i in numbers]
    
    def dab(self, number):
        if number in self.numbers:
            self.marked[self.numbers.index(number)] = True
            
    def is_winning(self):
        for num in range(5):
            if not False in self.marked[num*5:num*5+5]:
                return True
            if not False in self.marked[num::5]:
                return True
        return False
    
    def score(self):
        return sum([n for m,n in zip(self.marked, self.numbers) if not m])
    
    def __eq__(self, other):
        if isinstance(other,Board):
            return self.numbers == other.numbers
        return NotImplemented
    
    def __repr__(self):
        return f"Board(numbers={self.numbers}, marked={self.marked})"
    
    def __str__(self):
        s = "\n"
        for y in range(5):
            for x in range(5):
                if self.marked[y*5+x]: 
                    s+=f"*{self.numbers[y*5+x]:2}*"
                else:
                    s+=f" {self.numbers[y*5+x]:2} "
            s+="\n"
        return s
    
test_board = Board([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25])
assert test_board.score() == 325 
test_board.dab(3)
assert test_board.marked[0] == False
assert test_board.marked[3] == False
assert test_board.is_winning() == False
test_board.dab(8)
test_board.dab(13)
test_board.dab(18)
test_board.dab(23)
assert test_board.is_winning() == True
assert test_board.score() == 260 

That's a Board class, it lets us mark numbers, it keeps track of the marked numbers.

There's two clever bits of code in here that you might not have seen before though.

Firstly, there's a range that uses a step.  When you access a number inside an list you use `list[n]`.  But sometimes you want a range rather than just one number.  You can use `list[n:m]` to get you all the list from index n to index m-1 (it doesn't include the last index for ... reasons, just trust me).  You can shorten this if you want all the way from the beginning or all the way to the end, so `list[n:]` and `list[:m]` can be handy (we'll use this shortly on the input lines).  

But there's another thing, the step. By default, when you say `list[m:n]` it means every digit from m to n.  But you can also tell it to step by more than one, so `list[m:n:2]` will get you every other number from a list.  In our case we use `self.marked[num::5]` to get us every 5th number starting from `number` (0 to 5).  That's basically a column.

Secondly, I've used another list comprehension feature, the filter clause.  For a list comprehension `[item for item in list]` you can also put an if statement on it, for example `[item for item in list if item > 5]`.  We use that in the score to combine the list of numebrs and the list of marked with zip to give us a list of tuples like `[(False, 1), (False,2), (True, 3)...]`.  Instead of just item, I've used m,n to unpack the marked and number values, and then the if clause is returning only numbers that are not marked.  We then pass that list to sum, and viola, our scoring mechanism.

Final thing, I want to compare boards for testing, so I've implemented the special method `__eq__`.  This is called when you do `a == b`, and it does two things, if b is another Board, and if both a and b's list of numbers are the same, then it'll return true.  If b is any other class, it will return NotImplemented, and python will just compare object identity instead.  Finally I'm implemented __repr__ which is called when the test framework prints objects.

So let's try this weird parsing thing next.

We want to open our file, but now we're going to read it in a unique way.  We're going to read the first line and store those as numbers, and then we're going to read a card.  We want to store our cards as a single long list, so we'll read five lines, strip them of their linebreaks, and then join them together, with a space between each line.  We can then split them up by spaces and get 25 numbers out.

Let's go

In [3]:
def parse_file(lines):
    called_numbers = [int(num) for num in lines[0].split(",")]
    boards = []
    current_board = ""
    lines = lines[1:]
    for n in range(len(lines)//6):
        boardnums = " ".join([line for line in lines[6*n:(6*n)+6]])
        boards.append(Board([int(num) for num in boardnums.split()]))
    return called_numbers, boards

test_file = """7,4,9,5,11,17,23,2,0,14,21,24,10,16,13,6,15,25,12,22,18,20,8,19,3,26,1

22 13 17 11  0
 8  2 23  4 24
21  9 14 16  7
 6 10  3 18  5
 1 12 20 15 19

 3 15  0  2 22
 9 18 13 17  5
19  8  7 25 23
20 11 10 24  4
14 21 16 12  6

14 21 17 24  4
10 16 15  9 19
18  8 23 26 20
22 11 13  6  5
 2  0 12  3  7"""

test_calls, test_boards = parse_file(test_file.split('\n'))
assert [7,4,9,5,11,17,23,2,0,14,21,24,10,16,13,6,15,25,12,22,18,20,8,19,3,26,1] == test_calls
assert 3 == len(test_boards)
assert test_boards[0] == Board([22,13,17,11,0,8,2,23,4,24,21,9,14,16,7,6,10,3,18,5,1,12,20,15,19])
assert test_boards[2] == Board([14,21,17,24,4,10,16,15,9,19,18,8,23,26,20,22,11,13,6,5,2,0,12,3,7])

Yuck, that parsing was more complex than I thought.  We're joining together the lines, in 6's.  We're including the blank line because the split will eat all the extra whitespace and it's easier to take them in chunks of 6 than to work out how to handle the first or last board being different.

Ok, let's try dabbing our test boards and see if we can get teh same result as the example.  We're going to write a function that takes the call numbers and keeps dabbing until one of the boards tells it it's won, at which point it will return the winning board and we can then calculate it's score.

In [4]:
def dab_until_win(boards, numbers):
    for number in numbers:
        print(f"Calling {number}\n")
        for board in boards:
            board.dab(number)
            if board.is_winning():
                print(f"Win! Board: {board}\n")
                return board, number
    return None

test_calls, test_boards = parse_file(test_file.split('\n'))
winning_test_board, winning_test_number = dab_until_win(test_boards, test_calls)
assert winning_test_board == Board([14,21,17,24,4,10,16,15,9,19,18,8,23,26,20,22,11,13,6,5,2,0,12,3,7])
assert winning_test_number == 24
assert winning_test_board.score() == 188
print(24*188)

Calling 7

Calling 4

Calling 9

Calling 5

Calling 11

Calling 17

Calling 23

Calling 2

Calling 0

Calling 14

Calling 21

Calling 24

Win! Board: 
*14**21**17**24** 4*
 10  16  15 * 9* 19 
 18   8 *23* 26  20 
 22 *11* 13   6 * 5*
* 2** 0* 12   3 * 7*


4512


Blimey, wasn't expecting that to work first time!

Ok, let's try this bad boy on some real data and see what it tells us.

In [5]:
data = [line.strip() for line in open("day4.txt").readlines()]
calls, boards = parse_file(data)
winning_board, winning_number = dab_until_win(boards, calls)

print(f"winning board: {winning_board} at {winning_number} with score {winning_board.score()} equals {winning_board.score()*winning_number}")

Calling 83

Calling 5

Calling 71

Calling 61

Calling 88

Calling 55

Calling 95

Calling 6

Calling 0

Calling 97

Calling 20

Calling 16

Calling 27

Calling 7

Calling 79

Calling 25

Calling 81

Calling 29

Calling 22

Calling 52

Calling 43

Calling 21

Win! Board: 
 39 *55**88* 78  72 
* 6* 82 *52*  1  60 
 41  23 *97* 44  11 
  3  15 *21* 93  38 
 24  90 * 7* 80   2 


winning board: 
 39 *55**88* 78  72 
* 6* 82 *52*  1  60 
 41  23 *97* 44  11 
  3  15 *21* 93  38 
 24  90 * 7* 80   2 
 at 21 with score 796 equals 16716


That worked, although I'll admit now to 1 minor bug when I first wrote this.  First of all, I was still calling dab_until_win on the test data, which gave me the wrong total.  Seocndly, once I fixed that, I discovered I was trying to parse the day3 data, which was resulting in some very odd errors.  

Oh well, onto ...

## Part 2 Find out who loses

This is really interesting, because we're after a really specific condition.  As numbers are called, a board might be winning because it has a single line.  But when the next number is called, it might now win on a column, so it has two lines.  I was expecting something like this, keep playing until there's 2 lines or even a special pattern (I've seen christmas tree patterns on 5x5 bingo cards, as well as 4 corners plus center, and so on).

But in this case, even if a card has won, we keep playing until every card has won apart from one of them.  Even then we need to keep playing until that card finally wins.  We probably don't care about the other cards at that point, so I'm super tempted to start removing them from the list, that way we can wait until there's only 1 card left and then when it wins, we can return the card and the number as before.  Lets try that, it's just a new version of dab_until_win for us.

In [6]:
def dab_until_last_win(boards, numbers):
    for number in numbers:
        winning_boards = []
        for board in boards:
            board.dab(number)
            if board.is_winning():
                print(f"Win on {number} for board: {board}")
                if len(boards) == 1:
                    return board, number
                winning_boards.append(board)
        for board in winning_boards:
            boards.remove(board)
    return None

test_calls, test_boards = parse_file(test_file.split('\n'))
winning_test_board, winning_test_number = dab_until_last_win(test_boards, test_calls)
assert winning_test_board == Board([3, 15, 0, 2, 22, 9, 18, 13, 17, 5, 19, 8, 7, 25, 23, 20, 11, 10, 24, 4, 14, 21, 16, 12, 6])
assert winning_test_number == 13
assert winning_test_board.score() == 148
print(f"winning board: {winning_test_board} at {winning_test_number} with score {winning_test_board.score()} equals {winning_test_board.score()*winning_test_number}")

Win on 24 for board: 
*14**21**17**24** 4*
 10  16  15 * 9* 19 
 18   8 *23* 26  20 
 22 *11* 13   6 * 5*
* 2** 0* 12   3 * 7*

Win on 16 for board: 
 22  13 *17**11** 0*
  8 * 2**23** 4**24*
*21** 9**14**16** 7*
  6 *10*  3  18 * 5*
  1  12  20  15  19 

Win on 13 for board: 
  3  15 * 0** 2* 22 
* 9* 18 *13**17** 5*
 19   8 * 7* 25 *23*
 20 *11**10**24** 4*
*14**21**16* 12   6 

winning board: 
  3  15 * 0** 2* 22 
* 9* 18 *13**17** 5*
 19   8 * 7* 25 *23*
 20 *11**10**24** 4*
*14**21**16* 12   6 
 at 13 with score 148 equals 1924


Phew.

I ran into a problem I'd forgotten about there, removing items from a list while iterating through a list does really odd things to the iteration.  So I had to rework the process to iterate through the boards, adding winning boards to a list of boards to delete.

The second thing that I struggled with at this point was that I had made a mistake in the `is_winning` code.  The original cost checked for rows without multiplying.  Since I'd made the point of mentioning it, this was a stupid mistake to make.  More importantly, because somehow nothing so far had needed to know about the completion of rows 2-5, I didn't notice.

Finally, I added late a `__str__` function,which is used to render the boards as strings.  I used stars around numbers to indicate those that had been marked, making it easier to tell how well a board had been filled.  This is what tipped me off to my is_winning bug, when I discovered a board being marked as winning despite not having any lines completed!

In [7]:
data = [line.strip() for line in open("day4.txt").readlines()]
calls, boards = parse_file(data)
winning_board, winning_number = dab_until_last_win(boards, calls)

print(f"winning board: {winning_board} at {winning_number} with score {winning_board.score()} equals {winning_board.score()*winning_number}")

Win on 21 for board: 
 39 *55**88* 78  72 
* 6* 82 *52*  1  60 
 41  23 *97* 44  11 
  3  15 *21* 93  38 
 24  90 * 7* 80   2 

Win on 59 for board: 
 74  99 * 6*  4 *20*
*95**81**27**59**88*
 63  69  30 *25* 87 
 92  96  89  42  18 
 11  77  91   8  46 

Win on 96 for board: 
 94 *99* 67 *88* 82 
*96** 5**21**53**52*
 41  15  49 *35* 89 
 54  39  66  24  51 
  9 * 6* 62  33  70 

Win on 96 for board: 
 80  47 *53**81* 36 
 75 *35* 87  90  89 
 19 * 5* 56  28  26 
  8  44  77  31 *20*
*61**96**27**99**79*

Win on 51 for board: 
 78 *51* 69  47 *16*
 48 *55* 58  70  37 
* 7**59* 66 * 5* 76 
 94 *52* 82 *22* 10 
 13 *83**95* 24 *79*

Win on 51 for board: 
  1  66   8 * 6** 7*
 47 *96**25* 77  72 
 23 *22* 31  42  24 
*52**27**53**51**99*
*21* 65 *35* 84 * 5*

Win on 57 for board: 
*52**18* 32  56 *61*
 40 * 5* 48  64  62 
*22**57* 19  26  91 
 31 * 3**95**27* 87 
 74 *83* 75 *99* 73 

Win on 50 for board: 
*96* 45  75 *97* 94 
 68 *35*  9  30  67 
*25**88* 40  46  37 
 82 *79* 90  76 *55